# ***תרגיל 5 של הפרוייקט***

**Load Data and Basic Setup**

In [1]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("/content/train-filtered_question_level.csv")

# Remove duplicate questions
df = df.drop_duplicates(subset=["question"], keep="first")

# Extract text and difficulty levels
texts = df["question"].astype(str).tolist()
levels = df["level"].tolist()


**Balancing Dataset (Undersampling to Minority Class)**

In [4]:
import pandas as pd

# 1. נגדיר את גודל היעד לפי המחלקה הקטנה ביותר (Hard)
target_size = 15657

# 2. נבצע דגימה מכל מחלקה בנפרד
df_hard = df[df['level'] == 'hard']
# כאן אנחנו לא עושים sample כי זה כבר הגודל שאנחנו רוצים

df_medium_downsampled = df[df['level'] == 'medium'].sample(n=target_size, random_state=42)
df_easy_downsampled = df[df['level'] == 'easy'].sample(n=target_size, random_state=42)

# 3. נחבר את שלושתן יחד
df_balanced = pd.concat([df_hard, df_medium_downsampled, df_easy_downsampled])

# 4. נערבב את הדאטה (חשוב מאוד!)
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# בדיקת תוצאה
print("התפלגות חדשה:")
print(df_balanced['level'].value_counts())

התפלגות חדשה:
level
easy      15657
hard      15657
medium    15657
Name: count, dtype: int64


# **שלב 1**

# **א**

**Choosing Maximum Sequence Length (Documentation)**

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

# 1. הגדרת פרמטרים
VOCAB_SIZE = 20000

# 2. אתחול הטוקנייזר (יצירת האובייקט שהיה חסר)
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")

# 3. חילוץ הטקסטים מהדאטה המאוזן (ודא ששם העמודה נכון, נניח 'text')
texts = df_balanced['question'].astype(str).tolist()

# 4. התאמת הטוקנייזר על הטקסטים (שלב קריטי!)
tokenizer.fit_on_texts(texts)

# 5. המרה לרצפים של מספרים
sequences = tokenizer.texts_to_sequences(texts)

# עכשיו הסטטיסטיקות שלך יעבדו:
sequence_lengths = [len(seq) for seq in sequences]
avg_len = np.mean(sequence_lengths)
percentile_95 = np.percentile(sequence_lengths, 95)

print("Average sequence length:", round(avg_len, 2))
print("95th percentile length:", percentile_95)
print("Vocabulary size (actual):", len(tokenizer.word_index))

Average sequence length: 19.32
95th percentile length: 44.0
Vocabulary size (actual): 58011


**Padding and Truncation**

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Chosen maximum sequence length
MAX_SEQUENCE_LENGTH = int(percentile_95)

# Apply padding and truncation
X_padded = pad_sequences(
    sequences,
    maxlen=MAX_SEQUENCE_LENGTH,
    padding="post",
    truncating="post"
)

print("Final input shape:", X_padded.shape)


Final input shape: (46971, 44)


# **ב**